# Measuring centroids and position angles for the LoVoCCS sample

## Import Statements

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from astropy.units import Quantity, UnitConversionError
from astropy.cosmology import LambdaCDM
from shutil import rmtree
import os
from matplotlib import pyplot as plt

import xga
xga.NUM_CORES = 5
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/')[0]+'notebooks/xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.samples import ClusterSample

# This is a bit cheeky, but suppresses the warnings that XGA spits out (they are 
#  useful, but not when I'm trying to present this notebook on GitHub)
import warnings
warnings.filterwarnings('ignore')

# Set up a variable that controls how long individual XSPEC fits are allowed to run
timeout = Quantity(6, 'hr')

%matplotlib inline

## Setting up necessary directories

Here we ensure that the directories we need to store the outputs in have been created:

In [ ]:
if not os.path.exists("../../outputs/figures/positions/"):
    os.makedirs("../../outputs/figures/positions/")
    
if not os.path.exists("../../outputs/coordinates"):
    os.makedirs("../../outputs/coordinates/")
    
if not os.path.exists("../../outputs/cluster_visualisations/centroid_posangle_meas/"):
    os.makedirs("../../outputs/cluster_visualisations/centroid_posangle_meas/")

## Defining any useful functions

In [8]:
def centroid_shape_estimate(rel_src, ap_rad, cent_coord):
    
    cur_rt = src.get_combined_ratemaps()
    cont_mask = src.get_interloper_mask()
    
    if ap_rad is not None:
        ap_mask = src.get_custom_mask(ap_rad, remove_interlopers=False)
    else:
        ap_mask = np.ones(cur_rt.shape)
        
    tot_mask = ap_mask*cont_mask
    
    data_cp = cur_rt.data.copy()
    msk_data = data_cp*tot_mask
    norm_data = msk_data / msk_data.sum()
    
    arr_x, arr_y = np.meshgrid(np.arange(0, norm_data.shape[1], ), np.arange(0, norm_data.shape[0]))
    
    x_cen_pix = (norm_data*arr_x).sum()
    y_cen_pix = (norm_data*arr_y).sum()
    cen_pix = Quantity([x_cen_pix, y_cen_pix], 'pix')

#     cen_deg = cur_rt.coord_conv(cen_pix, 'deg')
    covar = np.zeros((2, 2))
    covar[0, 0] = (norm_img*np.square(arr_img_x - img_cen_pix[0])).sum()
    covar[1, 1] = (norm_img*np.square(arr_img_y - img_cen_pix[1])).sum()
    covar[0, 1] = (norm_img * (arr_img_x - img_cen_pix[0]) * (arr_img_y - img_cen_pix[1])).sum()
    covar[1, 0] = covar[0, 1]
    
#     np.arctan2(rec_x, rec_y) - start_ang
    
    # TODO Try to normalise this output + ensure that it is North of East (does my head in)
    theta = 0.5*np.arctan((2*covar[1, 0]) / (covar[0, 0]-covar[1, 1]))*(180/np.pi)
    
    eig_pl = ((covar[0, 0] + covar[1, 1]) + np.sqrt((covar[0, 0] - covar[0, 0])**2 + 4*covar[0, 1]**2)) / 2
    eig_mi = ((covar[0, 0] + covar[1, 1]) - np.sqrt((covar[0, 0] - covar[0, 0])**2 + 4*covar[0, 1]**2)) / 2

    x_s = np.sqrt(eig_pl)
    y_s = np.sqrt(eig_mi)
    
    return cen_pix, theta, x_s, y_s
    

## Reading in the sample

We read in the LoVoCCS sample relevant to the current work:

In [3]:
samp = pd.read_csv("../../sample_files/X-LoVoCCSI.csv")
samp['LoVoCCS_name'] = samp['LoVoCCSID'].apply(lambda x: "LoVoCCS-" + str(x))
samp

,LoVoCCSID,Name,start_ra,start_dec,MCXC_Redshift,MCXC_R500,MCXC_RA,MCXC_DEC,manual_xray_ra,manual_xray_dec,LoVoCCS_name
0,1,A2029,227.734300,5.745471,0.0766,1.3344,227.73000,5.720000,227.734300,5.745471,LoVoCCS-1
1,2,A401,44.740000,13.580000,0.0739,1.2421,44.74000,13.580000,NaN,NaN,LoVoCCS-2
2,4,A85,10.458750,-9.301944,0.0555,1.2103,10.45875,-9.301944,NaN,NaN,LoVoCCS-4
3,5,A3667,303.157313,-56.845978,0.0556,1.1990,303.13000,-56.830000,303.157313,-56.845978,LoVoCCS-5
4,7,A3827,330.480000,-59.950000,0.0980,1.1367,330.48000,-59.950000,NaN,NaN,LoVoCCS-7
...,...,...,...,...,...,...,...,...,...,...,...
61,121,A3128,52.466189,-52.580728,0.0624,0.8831,52.50000,-52.600000,52.466189,-52.580728,LoVoCCS-121
62,122,A1023,157.000000,-6.800000,0.1176,0.8553,157.00000,-6.800000,NaN,NaN,LoVoCCS-122
63,123,A3528,193.670000,-29.220000,0.0544,0.8855,193.67000,-29.220000,NaN,NaN,LoVoCCS-123
64,131,A761,137.651250,-10.581111,0.0916,0.8627,137.65125,-10.581111,NaN,NaN,LoVoCCS-131


In [6]:
# TODO OBVIOUSLY REMOVE THIS
samp = samp.sample(10, random_state=907)
samp

,LoVoCCSID,Name,start_ra,start_dec,MCXC_Redshift,MCXC_R500,MCXC_RA,MCXC_DEC,manual_xray_ra,manual_xray_dec,LoVoCCS_name
9,13,A3112,49.490000,-44.240000,0.0752,1.1288,49.490000,-44.240000,NaN,NaN,LoVoCCS-13
26,41B,A1750North,202.793430,-1.726838,0.0852,0.9968,202.707917,-1.872778,202.793430,-1.726838,LoVoCCS-41B
65,134,A3825,329.604682,-60.413676,0.0750,0.8685,329.610000,-60.400000,329.604682,-60.413676,LoVoCCS-134
2,4,A85,10.458750,-9.301944,0.0555,1.2103,10.458750,-9.301944,NaN,NaN,LoVoCCS-4
16,26,A2426,333.635833,-10.371667,0.0980,1.0442,333.635833,-10.371667,NaN,NaN,LoVoCCS-26
58,99,A3532,194.333566,-30.362907,0.0554,0.9201,194.320417,-30.376944,194.333566,-30.362907,LoVoCCS-99
55,93B,A3395South,96.687844,-54.539092,0.0506,0.9298,96.810000,-54.470000,96.687844,-54.539092,LoVoCCS-93B
29,45,A2941,26.259583,-53.013889,0.1168,0.9680,26.259583,-53.013889,NaN,NaN,LoVoCCS-45
6,10,A1651,194.840000,-4.190000,0.0845,1.1252,194.840000,-4.190000,NaN,NaN,LoVoCCS-10
31,46B,A2440North,336.063345,-1.516853,0.0906,0.9716,335.970000,-1.640000,336.063345,-1.516853,LoVoCCS-46B


## Defining an XGA ClusterSample

In [ ]:
srcs = ClusterSample(samp['start_ra'].values, samp['start_dec'].values, samp['MCXC_Redshift'].values, 
                     samp['LoVoCCS_name'].values, r500=Quantity(samp['MCXC_R500'].values, 'Mpc'), use_peak=False, 
                     clean_obs=False)
srcs.info()

## 